# Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
VOCAB_PATH = '/content/drive/MyDrive/ASR-Praktikum/vocab.pth'
MODEL_PATH = '/content/drive/MyDrive/ASR-Praktikum/final_model.pth'

# Karaok AI

In this notebook you can run forced alignment for youtube music videos. The results are displayed on a website using dash. The website provides an embedding of the youtube video and the highlighted transcript based on the timestamp of the video.

In [ ]:
!pip install pydub youtube-dl spleeter

In [ ]:
import IPython
import os
import pandas as pd
import re
import sys
import shutil
import torch
import torchaudio

from dataclasses import dataclass
from io import BytesIO

from pydub import AudioSegment

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
use_spleeter = True

# Download Model

We are using Pytorchs' WAV2VEC2_ASR_BASE_960H pre-trained on 960 hours of unlabeled audio from LibriSpeech dataset [Panayotov et al., 2015] (the combination of “train-clean-100”, “train-clean-360”, and “train-other-500”), and fine-tuned for ASR on the same audio with the corresponding transcripts.

In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
labels = bundle.get_labels()
model = bundle.get_model().to(device)
dictionary = {c: i for i, c in enumerate(labels)}

# Download YouTube Video

In [ ]:
if use_spleeter:
  from spleeter.separator import Separator
  # Initialize the separator
  separator = Separator('spleeter:2stems')

In [ ]:
def get_wave(aud):
  aud = aud.set_channels(1)
  aud = aud.get_array_of_samples()
  wave = torch.tensor(aud, dtype = torch.float)
  wave = torch.reshape(wave, (1,wave.shape[0]))

  return wave

In [ ]:
def get_wav_sr_from_yt_video_id(video_id):
    # Download the video using youtube-dl
    os.system("youtube-dl --extract-audio --audio-format wav --audio-quality 0 -o '%(id)s.%(ext)s' https://youtu.be/{}".format(video_id))

    file_path = "{}.wav".format(video_id)
    audio_path = file_path

    if use_spleeter:
      separator.separate_to_file("/content/{}.wav".format(video_id), "/content/")
      audio_path = "/content/{}/vocals.wav".format(video_id)

    # Load the audio file using pydub
    audio = AudioSegment.from_file(audio_path, format="wav")

    waveform = get_wave(audio)
    sr = audio.frame_rate

    # Delete file
    if os.path.isfile(file_path):
        os.remove(file_path)
    else:
        print("{} does not exist.".format(file_path))

    try:
        shutil.rmtree("/content/{}".format(video_id))
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
        
    # Resample
    if sr != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, bundle.sample_rate)

    return waveform, sr

In [ ]:
def clean_lyrics(lyrics):
    lyrics = re.sub(r"\[.*?\]", "", lyrics, flags=re.MULTILINE)
    lyrics = re.sub(r"’", "'", lyrics)
    lyrics = re.sub(r"[^a-zA-Z'’|-]|\s", "|", lyrics)
    return lyrics.upper()

# Forced Alignment
Based on Pytorchs' FORCED ALIGNMENT WITH WAV2VEC2.

## Get Emission of model

In [ ]:
def calculate_emission(waveform):
    torch.cuda.empty_cache()    
    
    length = waveform.shape[1]
    chunks = []
    amount_chunks = 10
    chunks_length = length//amount_chunks
    for i in range(amount_chunks):
        with torch.inference_mode():
            emissions, _ = model(waveform[:, i * chunks_length: min(length, (i + 1) * chunks_length)].to(device))
            emissions = torch.log_softmax(emissions, dim=-1)
            chunks.append(emissions)

    return torch.cat(chunks, dim=1)[0].cpu().detach()

## Generate alignment probability (trellis)

In [ ]:
def get_tokens(transcript):
    return [dictionary[c] for c in transcript]

In [ ]:
def get_trellis(emission, tokens, blank_id=0):
    num_frame = emission.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(emission[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + emission[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + emission[t, tokens],
        )
    return trellis

## Find the most likely path (backtracking)



In [ ]:
@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


def backtrack(trellis, emission, tokens, blank_id=0):
    # Note:
    # j and t are indices for trellis, which has extra dimensions
    # for time and tokens at the beginning.
    # When referring to time frame index `T` in trellis,
    # the corresponding index in emission is `T-1`.
    # Similarly, when referring to token index `J` in trellis,
    # the corresponding index in transcript is `J-1`.
    j = trellis.size(1) - 1
    t_start = torch.argmax(trellis[:, j]).item()

    path = []
    for t in range(t_start, 0, -1):
        # 1. Figure out if the current position was stay or change
        # Note (again):
        # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
        # Score for token staying the same from time frame J-1 to T.
        stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
        # Score for token changing from C-1 at T-1 to J at T.
        changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]

        # 2. Store the path with frame-wise probability.
        prob = emission[t - 1, tokens[j - 1] if changed > stayed else 0].exp().item()
        # Return token index and time index in non-trellis coordinate.
        path.append(Point(j - 1, t - 1, prob))

        # 3. Update the token
        if changed > stayed:
            j -= 1
            if j == 0:
                break
    else:
        raise ValueError("Failed to align")
    return path[::-1]

In [ ]:
# Merge the labels
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start}, {self.end})"

    @property
    def length(self):
        return self.end - self.start

    def __hash__(self):
        return hash((self.label, self.start, self.end, self.score))

    def __eq__(self, other):
        if not isinstance(other, Segment):
            return False
        return (self.label, self.start, self.end, self.score) == (other.label, other.start, other.end, other.score)



def merge_repeats(path, transcript):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments

In [ ]:
# Merge words
def merge_words(segments, ratio, sr, separator="|"):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)

                x0 = int(ratio * segments[i1].start)
                x1 = int(ratio * segments[i2 - 1].end)
                start = x0 / sr
                end = x1 / sr

                words.append(Segment(word, start, end, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words

## Execute forced alignment




In [ ]:
def execute(audio, transcript):
    transcript = clean_lyrics(transcript)
    emission = calculate_emission(audio)
    tokens = get_tokens(transcript)
    trellis = get_trellis(emission, tokens)
    path = backtrack(trellis, emission, tokens)
    segments = merge_repeats(path, transcript)

    ratio = audio.size(1) / (trellis.size(0) - 1)

    word_segments = merge_words(segments, ratio=ratio, sr = bundle.sample_rate)
    return emission, tokens, trellis, path, segments, word_segments

In [ ]:
def execute_with_id(video_id, transcript):
    waveform, sr = get_wav_sr_from_yt_video_id(video_id)
    return execute(waveform, transcript)

# Genre Prediction

In [ ]:
from torch import nn
from torch.nn import functional as F

embed_len = 50
hidden_dim = 75
n_layers=1
num_class = 7

class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_class)
        self.dropout = nn.Dropout(0.3)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        hidden, carry = torch.randn(n_layers, len(X_batch), hidden_dim), torch.randn(n_layers, len(X_batch), hidden_dim)
        output, (hidden, carry) = self.lstm(embeddings, (hidden.to(device), carry.to(device)))
        output = self.dropout(output)
        return self.linear(output[:,-1])

In [ ]:
import torch
from torch.optim import Adam
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab = torch.load(VOCAB_PATH)
lstm_classifier = LSTMClassifier().to(device)
optimizer = Adam(lstm_classifier.parameters(), lr=0.001, weight_decay=1e-5)

checkpoint = torch.load(MODEL_PATH)
lstm_classifier.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
lstm_classifier.eval()

from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

text_pipeline = lambda x: vocab(tokenizer(x))

import regex as re
def clean_lyrics_2(lyrics):
    lyrics = re.sub(r"[^A-Za-z0-9']+", " ", lyrics, flags=re.MULTILINE)
    lyrics = re.sub(r"(?<=[a-zA-Z0-9]) (?=['])|(?<=[']) (?=[a-zA-Z0-9])", "", lyrics, flags=re.MULTILINE)
    lyrics = re.sub('\s+',' ',lyrics)
    return lyrics.lower().lstrip()

decode_labels = {0:"Pop",1:"R&B",2:"Hip Hop",3:"Rock",4:"Indie",5:"Country",6:"Heavy Metal"}

In [ ]:
def predict_genre(lyrics):
  pred = lstm_classifier(torch.tensor(text_pipeline(clean_lyrics_2(lyrics))).unsqueeze(0).to(device))
  label = decode_labels[F.softmax(pred, dim=-1).argmax(dim=-1).item()]
  return label

# Website

In [ ]:
!pip install jupyter-dash
!pip install dash-player

In [ ]:
import re

In [ ]:
def extract_video_id(link):
    regExp = re.compile(r'^.*(youtu.be\/|v\/|u\/\w\/|embed\/|watch\?v=|&v=)([^#&?]*).*')
    match_id = regExp.match(link)
    if match_id:
        video_id = match_id.group(2)
        if len(video_id) == 11:
            return video_id 
    return None

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_player

In [ ]:
import time

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
app.layout = html.Div([
    html.H1("AI Karaoke"),
    # Yt link
    html.Div([
        dcc.Input(id="input_yt", placeholder="Input Youtube Link", style={'width': '600px', 'margin-right': '5px'}),
        html.Button('Submit', id='btn_submit', n_clicks=0, style={'margin': '0px 5px'}),
        html.Button('Reset', id='btn_reset', n_clicks=0, style={'margin': '0px 5px'}),
        dcc.Loading(
            id="loading-1",
            children=html.Div(id="loading-output", style={'display': 'none'})
        ),
        html.Div(id="initial_message", children="Enter a YouTube link and transcript and press submit to load video", style={'display': 'block'}),
        html.Div(id="invalid_link_div", children="Invalid YouTube link", style={'color': 'red', 'display': 'none'}),
        html.Div(id="no_transcript_div", children="Please enter a transcript", style={'color': 'red', 'display': 'none'}),
    ], style={'margin-bottom': '5px'}),

    # video and transcript
    html.Div([
        dash_player.DashPlayer(id="player", url="", controls=True, width="70%", height="80%", style={'display': 'inline-block', 'margin-right': '10px'}),
        # TODO change input as you need
        dcc.Textarea(id='input_transcript', value="", placeholder="Input transcript", style={'width': '29%', 'height': '80%', 'display': 'inline-block'}),
        html.Div(id='output_transcript', children=[
            html.Div(id='pre_transcript', style={'display': 'inline'}), 
            html.Span(id='pre_highlight_word', style={'display': 'inline', 'background-color': '#ffffb3'}), 
            html.Span(id='highlight_word', style={'display': 'inline', 'background-color': 'orange'}), 
            html.Span(id='post_highlight_word', style={'display': 'inline', 'background-color': '#ffff99'}), 
            html.Div(id='post_transcript', style={'display': 'inline'})
            ], style={'width': '29%', 'height': '80%', 'display': 'none', 'overflow-y': 'auto'})
    ], style={'height': '100vh', 'width': '100%', 'display': 'flex'}),

    html.Div([
        html.Div(id="genre_div", children="Predicted Genre:", style={'display': 'inline-block', 'margin-right': '5px'}),
        html.Div(id="genre_class", children="", style={'display': 'inline-block', 'font-weight': 'bold'}),
    ]),

    # current video timestamp, only for debugging
    html.Div(id="div_current_time", style={"margin": "10px 0px"}),
    dcc.Interval(id='interval', interval=2, n_intervals=0),
    dcc.Store(id='clientside-store-data')
])

In [ ]:
submit_clicks = 0
reset_clicks = 0


@app.callback(
    [Output('player', 'url'),
    Output('initial_message', 'style'),
    Output('invalid_link_div', 'style'),
    Output('no_transcript_div', 'style'),
    Output('input_transcript', 'style'),
    Output('output_transcript', 'style'),
    Output('clientside-store-data', 'data'),
    Output('input_yt', 'value'),
    Output('input_transcript', 'value'),
     Output("loading-output", "children")],
    [Input('btn_submit', 'n_clicks'),
    Input('btn_reset', 'n_clicks')],
    [State('input_yt', 'value'),
    State('input_transcript', 'value')]
)
def embed_video(btn_submit_n_clicks, btn_reset_n_clicks, link, transcript):
    global submit_clicks
    global reset_clicks

    initial_message_style = {'display': 'block'}
    invalid_style = {'color': 'red', 'display': 'none'}
    no_transcript = {'color': 'red', 'display': 'none'}

    input_style = {'width': '29%', 'height': '80%', 'display': 'inline-block', 'overflow-y': 'auto', 'border-style': 'solid', 'border-width': '1px'}
    output_style = {'width': '29%', 'height': '80%', 'display': 'none', 'overflow-y': 'auto', 'border-style': 'solid', 'border-width': '1px'}

    dict_words = {}
    url = ""
    if btn_submit_n_clicks > submit_clicks:
        submit_clicks += 1
        # TODO maybe add loading bar
        video_id = extract_video_id(link)
        if not video_id or not transcript:
            initial_message_style = {'display': 'none'}
            if not video_id:
                invalid_style = {'color': 'red', 'display': 'block'}
            if not transcript:
                no_transcript = {'color': 'red', 'display': 'block'}
        else:
            url = link
            input_style, output_style = output_style, input_style
            waveform, sr = get_wav_sr_from_yt_video_id(video_id)
            _, _, trellis, _, _, word_segments = execute(waveform, transcript)
            dict_words = pd.DataFrame([vars(f) for f in word_segments]).to_dict('records')
    
    if btn_reset_n_clicks > reset_clicks:
        reset_clicks += 1
        transcript = ""

    return url, initial_message_style, invalid_style, no_transcript, input_style, output_style, dict_words, url, transcript, None

app.clientside_callback(
    """
    function highlightWords(n_intervals, current_time, input, data) {
        let pre_transcript = "";
        let pre_highlight_word = "";
        let highlight_word = "";
        let post_highlight_word = "";
        let post_transcript = "";
        var listLength = data.length;
        for (var i = 0; i < listLength; i++) {
            let word = data[i];
            if (current_time != null) {
                start = word['start'];
                end = word['end'];
                if (current_time <= start - 1){
                    post_transcript += ' ' + word['label'];
                }
                if (current_time >= start - 1 && current_time <= start){
                    post_highlight_word += ' ' + word['label'];
                }
                if (current_time >= start && current_time <= end){
                    highlight_word += word['label'];
                }
                if (current_time >= end && current_time <= end + 1){
                    pre_highlight_word += word['label'] + ' ';
                }
                if (current_time >= end +1){
                    pre_transcript += word['label'] + ' ';
                }
            }
        }
        return [pre_transcript, pre_highlight_word, highlight_word, post_highlight_word, post_transcript];
    }
    """,
    [Output('pre_transcript', 'children'),
    Output('pre_highlight_word', 'children'),
    Output('highlight_word', 'children'),
    Output('post_highlight_word', 'children'),
    Output('post_transcript', 'children')],
    [Input('interval', 'n_intervals')],
    [State('player', 'currentTime'),
     State('input_transcript', 'value'),
     State('clientside-store-data', 'data')]
)

@app.callback(
    Output('genre_class', 'children'),
    Input('btn_submit', 'n_clicks'),
    State('input_transcript', 'value')
)
def display_genre(n_clicks, transcript):
    return predict_genre(transcript)

In [ ]:
# click link to open website in new tab
if __name__ == '__main__':
    app.run_server(mode='inline')
    # app.run_server(debug=True)